# Operations and Supply Chain Analytics - Assignment 1

**Due:** 5:00pm Eastern Time, March 31 (Sunday)  
**Submission:** Submit both a write-up and your Excel files electronically on Canvas.  
**Student:** Rodolfo Lerma

## Introduction

This Jupyter Notebook presents my solutions to the first assignment for the "Operations and Supply Chain Analytics" course. The assignment encompasses a range of problems designed to test our understanding and application of supply chain and operations analytics concepts, including office location optimization, production network planning, transport optimization, and inventory management. The solutions herein leverage Python and its libraries, such as PuLP for linear programming, to model and solve the given problems.

## Table of Contents

### Problem 1: Office Location and Consultant Assignment
  - In this problem, SC Consulting must decide on the optimal location of its home offices and the number of consultants to assign to each office, with the goal of minimizing costs. The problem considers annual fixed costs of locating offices, expected number of trips to each state, and travel costs from each potential site.

### Problem 2: Production Network for StayFresh
  - StayFresh, a refrigerator manufacturer in India, needs to plan its production network evolution over the next five years to accommodate anticipated demand growth. The problem involves deciding on capacity expansion at existing and potential plant sites, considering variable production and transport costs, as well as one-time capacity expansion costs.

### Problem 3: Optimal Engine Load
  - Harley-Davidson transports engines from its assembly plant in Milwaukee to its motorcycle assembly plant in Pennsylvania using trucks. The problem requires determining the optimal number of engines to load onto each truck, considering transportation costs, engine costs, and annual holding costs.

### Problem 4: JIT Manufacturing Impact
  - As part of a just-in-time (JIT) manufacturing initiative, Harley-Davidson has reduced the number of engines loaded on each truck. The problem asks to analyze the impact of this decision on annual inventory costs and determine the optimal truck cost for the given load of engines.

### Problem 5: Green Thumb Manufacturing Decision
  - Green Thumb, a manufacturer of lawn care equipment, has introduced a new product with a normally distributed demand. The problem involves determining the optimal number of units to manufacture, considering manufacturing costs, selling price, holding costs, and salvage value of unsold units. The goal is to maximize expected profit and estimate the average number of customers turned away due to stockouts.

## Problem 1: Office Location and Consultant Assignment

SC Consulting is tasked with deciding on the location of its home offices based on the annual fixed cost, expected number of trips, and travel costs. This section models and solves for the optimal office locations and number of consultants to minimize the overall cost.

In [2]:
import pulp
import pandas as pd

In [3]:
# Potential office locations with their fixed costs
locations = ["Los Angeles", "Tulsa", "Denver", "Seattle"]
fixed_costs = {"Los Angeles": 165428, "Tulsa": 131230, "Denver": 140000, "Seattle": 145000}

# Total number of trips required per state
trips_per_state = {
    "Washington": 40, "Oregon": 35, "California": 100, "Idaho": 25, "Nevada": 40,
    "Montana": 25, "Wyoming": 50, "Utah": 30, "Arizona": 50, "Colorado": 65,
    "New Mexico": 40, "North Dakota": 30, "South Dakota": 20, "Nebraska": 30,
    "Kansas": 40, "Oklahoma": 55
}

# Cost dictionaries by location
cost_from_location = {
    "Los Angeles": {"Washington": 150, "Oregon": 150, "California": 75, "Idaho": 150, "Nevada": 100, "Montana": 175, "Wyoming": 150, "Utah": 150, "Arizona": 75, "Colorado": 150, "New Mexico": 125, "North Dakota": 300, "South Dakota": 300, "Nebraska": 250, "Kansas": 250, "Oklahoma": 250},
    "Tulsa": {"Washington": 250, "Oregon": 250, "California": 200, "Idaho": 200, "Nevada": 200, "Montana": 175, "Wyoming": 175, "Utah": 150, "Arizona": 200, "Colorado": 125, "New Mexico": 125, "North Dakota": 200, "South Dakota": 175, "Nebraska": 100, "Kansas": 75, "Oklahoma": 25},
    "Denver": {"Washington": 200, "Oregon": 200, "California": 150, "Idaho": 125, "Nevada": 125, "Montana": 125, "Wyoming": 100, "Utah": 100, "Arizona": 100, "Colorado": 25, "New Mexico": 75, "North Dakota": 150, "South Dakota": 125, "Nebraska": 125, "Kansas": 75, "Oklahoma": 125},
    "Seattle": {"Washington": 25, "Oregon": 75, "California": 125, "Idaho": 125, "Nevada": 150, "Montana": 125, "Wyoming": 150, "Utah": 200, "Arizona": 250, "Colorado": 250, "New Mexico": 300, "North Dakota": 200, "South Dakota": 200, "Nebraska": 250, "Kansas": 300, "Oklahoma": 300},
}

In [3]:
# Problem A
problem_a = pulp.LpProblem("SC_Consulting_Office_Location_A", pulp.LpMinimize)

# Decision variables for Problem A
consultants_a = pulp.LpVariable.dicts("Consultants_A", locations, lowBound=0, cat='Integer')
offices_open_a = pulp.LpVariable.dicts("Offices_Open_A", locations, cat='Binary')
trips_a = pulp.LpVariable.dicts("Trips_A", (locations, trips_per_state.keys()), lowBound=0, cat='Integer')

# Objective function for Problem A
total_fixed_cost_a = pulp.lpSum([fixed_costs[location] * offices_open_a[location] for location in locations])
total_travel_cost_a = pulp.lpSum([
    cost_from_location[location][state] * trips_a[location][state]
    for location in locations for state in trips_per_state
])
problem_a += total_fixed_cost_a + total_travel_cost_a

# Constraints for Problem A
for state in trips_per_state:
    problem_a += pulp.lpSum([trips_a[location][state] for location in locations]) == trips_per_state[state]

for location in locations:
    problem_a += pulp.lpSum([trips_a[location][state] for state in trips_per_state]) <= 25 * consultants_a[location]
    problem_a += consultants_a[location] <= 1000000 * offices_open_a[location]  # Large number to allow unlimited consultants if office is open

# Solve Problem A
problem_a.solve()

# Print the results for Problem A
print("Problem A - Optimal Number of Consultants and Location:")
for location in locations:
    if pulp.value(offices_open_a[location]) == 1:
        print(f"{location}: {pulp.value(consultants_a[location])} consultants")

total_cost_a = pulp.value(total_fixed_cost_a + total_travel_cost_a)
print(f"Problem A - Total Annual Cost: ${total_cost_a:,.2f}")

Problem A - Optimal Number of Consultants and Location:
Denver: 27.0 consultants
Problem A - Total Annual Cost: $219,500.00


In [5]:
# Problem B
problem_b = pulp.LpProblem("SC_Consulting_Office_Location_B", pulp.LpMinimize)

# Decision variables for Problem B
consultants_b = pulp.LpVariable.dicts("Consultants_B", locations, lowBound=0, cat='Integer')
offices_open_b = pulp.LpVariable.dicts("Offices_Open_B", locations, cat='Binary')
trips_b = pulp.LpVariable.dicts("Trips_B", (locations, trips_per_state.keys()), lowBound=0, cat='Integer')

# Objective function for Problem B
total_fixed_cost_b = pulp.lpSum([fixed_costs[location] * offices_open_b[location] for location in locations])
total_travel_cost_b = pulp.lpSum([
    cost_from_location[location][state] * trips_b[location][state]
    for location in locations for state in trips_per_state
])
problem_b += total_fixed_cost_b + total_travel_cost_b

# Constraints for Problem B
for state in trips_per_state:
    problem_b += pulp.lpSum([trips_b[location][state] for location in locations]) == trips_per_state[state]

for location in locations:
    problem_b += pulp.lpSum([trips_b[location][state] for state in trips_per_state]) <= 25 * consultants_b[location]
    problem_b += consultants_b[location] <= 10 * offices_open_b[location]  # Limit the maximum number of consultants in each office to 10

# Solve Problem B
problem_b.solve()

# Print the results for Problem B
print("Problem B - Optimal Number of Consultants and Location:")
for location in locations:
    if pulp.value(offices_open_b[location]) == 1:
        print(f"{location}: {pulp.value(consultants_b[location])} consultants")

# Create an empty DataFrame to store the trips per home office to each state
trips_df = pd.DataFrame(columns=["State", "Total # of trips"] + locations)

# Fill the DataFrame with the values
for state in trips_per_state:
    row = {"State": state, "Total # of trips": trips_per_state[state]}
    for location in locations:
        trips = pulp.value(trips_b[location][state])
        row[location] = trips if trips > 0 else "-"
    trips_df = pd.concat([trips_df, pd.DataFrame([row])], ignore_index=True)

# Print the DataFrame
print("\nProblem B - Trips per Home Office to Each State:")
print(trips_df)

total_cost_b = pulp.value(total_fixed_cost_b + total_travel_cost_b)
print(f"\nProblem B - Total Annual Cost: ${total_cost_b:,.2f}")

Problem B - Optimal Number of Consultants and Location:
Tulsa: 7.0 consultants
Denver: 10.0 consultants
Seattle: 10.0 consultants

Problem B - Trips per Home Office to Each State:
           State Total # of trips Los Angeles Tulsa Denver Seattle
0     Washington               40           -     -      -    40.0
1         Oregon               35           -     -      -    35.0
2     California              100           -     -      -   100.0
3          Idaho               25           -     -      -    25.0
4         Nevada               40           -     -   15.0    25.0
5        Montana               25           -     -      -    25.0
6        Wyoming               50           -     -   50.0       -
7           Utah               30           -     -   30.0       -
8        Arizona               50           -     -   50.0       -
9       Colorado               65           -     -   65.0       -
10    New Mexico               40           -  20.0   20.0       -
11  North Dakota

C:\Users\rodol\AppData\Local\Temp\ipykernel_14252\2547427310.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trips_df = pd.concat([trips_df, pd.DataFrame([row])], ignore_index=True)


## Problem 2: Production Network for StayFresh

In this section, we address the StayFresh supply chain optimization problem, focusing on efficiently distributing refrigerators from manufacturing plants in Mumbai and Chennai to meet the regional demands across India. The challenge involves planning the production and potential expansion of facilities to accommodate a forecasted increase in demand while minimizing the associated costs.

**Objectives**

The primary objective is to minimize the total cost, which includes:
- The production and transportation costs from each plant to each regional market.
- The costs associated with expanding the production capacity at each plant location.

**Decision Variables**

- **Production Variables:** Represent the quantity of refrigerators produced at each plant for each regional market, across the five-year planning horizon.
- **Expansion Variables:** Indicate whether or not the company decides to expand the capacity at each plant location by either 150,000 or 300,000 units, in each year of the planning period.

**Constraints**

1. **Demand Satisfaction:** The total number of refrigerators produced and transported must meet or exceed the demand in each regional market, for each year.
2. **Capacity Limitation:** The production at each plant cannot exceed its available capacity, taking into account any expansions.


In [6]:
import pulp

# Define the problem
prob = pulp.LpProblem("StayFresh_Problem", pulp.LpMinimize)

# Define the decision variables
locations = ["Chennai", "Delhi", "Kolkata", "Mumbai"]
regions = ["North", "East", "West", "South"]
years = range(1, 6)

production = pulp.LpVariable.dicts("Production", (locations, regions, years), lowBound=0, cat='Integer')
expansion = pulp.LpVariable.dicts("Expansion", (locations, years), lowBound=0, cat='Integer')

# Define the objective function
costs = {
    "Chennai": {"North": 20, "East": 19, "West": 17, "South": 15},
    "Delhi": {"North": 15, "East": 18, "West": 17, "South": 20},
    "Kolkata": {"North": 18, "East": 15, "West": 20, "South": 19},
    "Mumbai": {"North": 17, "East": 20, "West": 15, "South": 17}
}

expansion_costs = {150: 2000, 300: 3400}
discount_factor = 0.20

total_cost = pulp.lpSum(
    [
        costs[loc][reg] * production[loc][reg][year] * (1 / (1 + discount_factor) ** (year - 1))
        for loc in locations for reg in regions for year in years
    ]
    +
    [
        expansion_costs[150] * expansion[loc][year] * (1 / (1 + discount_factor) ** (year - 1))
        + expansion_costs[300] * expansion[loc][year] * (1 / (1 + discount_factor) ** (year - 1))
        for loc in locations for year in years
    ]
)

prob += total_cost

# Define the constraints
initial_demand = {"North": 100, "East": 50, "West": 150, "South": 150}
growth_rate = 0.20
initial_capacity = {"Chennai": 300, "Delhi": 0, "Kolkata": 0, "Mumbai": 300}

for year in years:
    for reg in regions:
        prob += pulp.lpSum(production[loc][reg][year] for loc in locations) >= initial_demand[reg] * (1 + growth_rate) ** (year - 1)

for year in years:
    for loc in locations:
        prob += pulp.lpSum(production[loc][reg][year] for reg in regions) <= initial_capacity[loc] + pulp.lpSum(
            expansion[loc][y] * 150 + expansion[loc][y] * 300 for y in range(1, year + 1)
        )

# Solve the problem
prob.solve()

# Print the results
print("Status:", pulp.LpStatus[prob.status])
for year in years:
    print(f"\nYear {year}")
    
    total_demand = sum(initial_demand[reg] * (1 + growth_rate) ** (year - 1) for reg in regions)
    print(f"\nDemand: {total_demand:.1f}")
    
    print("\nProduction")
    total_production = 0
    for loc in locations:
        for reg in regions:
            if production[loc][reg][year].varValue > 0:
                print(f"Plant {loc}, Region {reg}: {production[loc][reg][year].varValue:.1f}")
                total_production += production[loc][reg][year].varValue
    print(f"\nTOTAL Production: {total_production:.1f}")
    
    print("\nExpansion (if any)")
    for loc in locations:
        for capacity in expansion_costs:
            if expansion[loc][year].varValue > 0:
                print(f"Plant {loc}, Expansion {capacity}: {expansion[loc][year].varValue:.1f}")
    
    total_cost_year = pulp.lpSum(
        costs[loc][reg] * production[loc][reg][year].varValue * (1 / (1 + discount_factor) ** (year - 1))
        for loc in locations for reg in regions
    )
    total_cost_year += pulp.lpSum(
        expansion_costs[150] * expansion[loc][year].varValue * (1 / (1 + discount_factor) ** (year - 1))
        + expansion_costs[300] * expansion[loc][year].varValue * (1 / (1 + discount_factor) ** (year - 1))
        for loc in locations
    )
    print(f"\nTOTAL Cost for Year {year}: {(total_cost_year.value())/1000:.2f} Million Rupees")

print(f"\nTotal cost (in millions of Rupees): {pulp.value(total_cost/1000):.2f}")

Status: Optimal

Year 1

Demand: 450.0

Production
Plant Chennai, Region East: 50.0
Plant Chennai, Region South: 150.0
Plant Mumbai, Region North: 100.0
Plant Mumbai, Region West: 150.0

TOTAL Production: 450.0

Expansion (if any)

TOTAL Cost for Year 1: 7.15 Million Rupees

Year 2

Demand: 540.0

Production
Plant Chennai, Region East: 60.0
Plant Chennai, Region South: 180.0
Plant Mumbai, Region North: 120.0
Plant Mumbai, Region West: 180.0

TOTAL Production: 540.0

Expansion (if any)

TOTAL Cost for Year 2: 7.15 Million Rupees

Year 3

Demand: 648.0

Production
Plant Chennai, Region South: 216.0
Plant Delhi, Region North: 144.0
Plant Delhi, Region East: 72.0
Plant Mumbai, Region West: 216.0

TOTAL Production: 648.0

Expansion (if any)
Plant Delhi, Expansion 150: 1.0
Plant Delhi, Expansion 300: 1.0

TOTAL Cost for Year 3: 10.65 Million Rupees

Year 4

Demand: 777.6

Production
Plant Chennai, Region South: 260.0
Plant Delhi, Region North: 173.0
Plant Delhi, Region East: 87.0
Plant Mumba

In [7]:
import pulp

def solve_stayfresh_problem(growth_rate):
    # Define the problem
    prob = pulp.LpProblem("StayFresh_Problem", pulp.LpMinimize)

    # Define the decision variables
    locations = ["Chennai", "Delhi", "Kolkata", "Mumbai"]
    regions = ["North", "East", "West", "South"]
    years = range(1, 6)

    production = pulp.LpVariable.dicts("Production", (locations, regions, years), lowBound=0, cat='Continuous')
    expansion = pulp.LpVariable.dicts("Expansion", (locations, years, [150, 300]), cat='Binary')

    # Define the objective function
    costs = {
        "Chennai": {"North": 20, "East": 19, "West": 17, "South": 15},
        "Delhi": {"North": 15, "East": 18, "West": 17, "South": 20},
        "Kolkata": {"North": 18, "East": 15, "West": 20, "South": 19},
        "Mumbai": {"North": 17, "East": 20, "West": 15, "South": 17}
    }

    expansion_costs = {150: 2000, 300: 3400}
    discount_factor = 0.20

    total_cost = pulp.lpSum(
        [
            costs[loc][reg] * production[loc][reg][year] * (1 / (1 + discount_factor) ** (year - 1)) 
            for loc in locations for reg in regions for year in years
        ]
        +
        [
            expansion_costs[capacity] * expansion[loc][year][capacity] * (1 / (1 + discount_factor) ** (year - 1))
            for loc in locations for year in years for capacity in [150, 300]
        ]
    )

    prob += total_cost

    # Define the constraints
    initial_demand = {"North": 100, "East": 50, "West": 150, "South": 150}
    initial_capacity = {"Chennai": 300, "Delhi": 0, "Kolkata": 0, "Mumbai": 300}

    for year in years:
        for reg in regions:
            prob += pulp.lpSum(production[loc][reg][year] for loc in locations) >= initial_demand[reg] * (1 + growth_rate) ** (year - 1)

    for year in years:
        for loc in locations:
            prob += pulp.lpSum(production[loc][reg][year] for reg in regions) <= initial_capacity[loc] + pulp.lpSum(
                expansion[loc][y][capacity] * capacity for y in range(1, year + 1) for capacity in [150, 300]
            )

    # Solve the problem
    prob.solve()

    # Print the results
    print(f"Optimal Solution for {growth_rate*100:.0f}% Growth Rate:")
    for year in years:
        print(f"\nYear {year}")
        
        total_demand = sum(initial_demand[reg] * (1 + growth_rate) ** (year - 1) for reg in regions)
        print(f"\nDemand: {total_demand:.0f}")
        
        print("\nProduction")
        total_production = 0
        for loc in locations:
            for reg in regions:
                if production[loc][reg][year].varValue > 0:
                    print(f"Plant {loc}, Region {reg}: {production[loc][reg][year].varValue:.0f}")
                    total_production += production[loc][reg][year].varValue
        print(f"\nTOTAL Production: {total_production:.0f}")
        
        print("\nExpansion (if any)")
        for loc in locations:
            for capacity in [150, 300]:
                if expansion[loc][year][capacity].varValue > 0:
                    print(f"Plant {loc}, Expansion {capacity}: {expansion[loc][year][capacity].varValue:.0f}")
        
        total_cost_year = pulp.lpSum(
            costs[loc][reg] * production[loc][reg][year].varValue * (1 / (1 + discount_factor) ** (year - 1))
            for loc in locations for reg in regions
        )
        total_cost_year += pulp.lpSum(
            expansion_costs[capacity] * expansion[loc][year][capacity].varValue * (1 / (1 + discount_factor) ** (year - 1))
            for loc in locations for capacity in [150, 300]
        )
        print(f"\nTOTAL Cost for Year {year}: {(total_cost_year.value())/1000:.2f} Million Rupees")

    print(f"\nTotal cost (in millions of Rupees): {pulp.value(total_cost/1000):.2f}")

# Solve the problem for different growth rates
growth_rates = [0.15, 0.20, 0.25]

for growth_rate in growth_rates:
    solve_stayfresh_problem(growth_rate)
    print("\n" + "-" * 50 + "\n")

Optimal Solution for 15% Growth Rate:

Year 1

Demand: 450

Production
Plant Chennai, Region East: 50
Plant Chennai, Region South: 150
Plant Mumbai, Region North: 100
Plant Mumbai, Region West: 150

TOTAL Production: 450

Expansion (if any)

TOTAL Cost for Year 1: 7.15 Million Rupees

Year 2

Demand: 518

Production
Plant Chennai, Region East: 58
Plant Chennai, Region South: 172
Plant Mumbai, Region North: 115
Plant Mumbai, Region West: 172

TOTAL Production: 518

Expansion (if any)

TOTAL Cost for Year 2: 6.85 Million Rupees

Year 3

Demand: 595

Production
Plant Chennai, Region East: 66
Plant Chennai, Region West: 31
Plant Chennai, Region South: 198
Plant Mumbai, Region North: 132
Plant Mumbai, Region West: 168

TOTAL Production: 595

Expansion (if any)

TOTAL Cost for Year 3: 6.61 Million Rupees

Year 4

Demand: 684

Production
Plant Chennai, Region South: 228
Plant Delhi, Region North: 152
Plant Delhi, Region East: 76
Plant Mumbai, Region West: 228

TOTAL Production: 684

Expansion

## Problem 3: Optimal Engine Load

In [8]:
# Given parameters
engine_cost = 500
holding_cost = 0.2
daily_demand = 300
truck_cost = 1000

# Calculate the economic order quantity (EOQ)
eoq = ((2 * daily_demand * truck_cost) / (engine_cost * holding_cost)) ** 0.5

# Round the EOQ to the nearest integer
optimal_load = round(eoq)

print(f"Harley should load {optimal_load} engines onto each truck.")

Harley should load 77 engines onto each truck.


In [4]:
import math

# Given data
annual_demand = 300 * 365  # 300 motorcycles per day, assuming 365 days per year
ordering_cost = 1000  # $1,000 per truck trip
unit_cost = 500  # $500 per engine
holding_cost_rate = 0.2  # 20% annual holding cost rate

# Calculate the annual holding cost per unit
annual_holding_cost = unit_cost * holding_cost_rate

# Calculate the Economic Order Quantity (EOQ)
eoq = math.sqrt((2 * annual_demand * ordering_cost) / annual_holding_cost)

# Print the result
print(f"Harley-Davidson should load {round(eoq)} engines onto each truck.")

Harley-Davidson should load 1480 engines onto each truck.


In [1]:
import math

# Given data
annual_demand = 300 * 365  # 300 motorcycles per day, assuming 365 days per year
ordering_cost = 1000  # $1,000 per truck trip
unit_cost = 500  # $500 per engine
holding_cost_rate = 0.2  # 20% annual holding cost rate

# Calculate the annual holding cost per unit
annual_holding_cost = unit_cost * holding_cost_rate

# Calculate the Economic Order Quantity (EOQ)
eoq = math.sqrt((2 * annual_demand * ordering_cost) / annual_holding_cost)

# Print the result
print(f"Harley-Davidson should load {round(eoq)} engines onto each truck.")

Harley-Davidson should load 1480 engines onto each truck.


## Problem 4: JIT Manufacturing Impact

In [9]:
# Given parameters
engine_cost = 500
holding_cost = 0.2
daily_demand = 300
truck_cost = 1000
jit_load = 100

# Calculate the annual inventory cost with EOQ
eoq = ((2 * daily_demand * truck_cost) / (engine_cost * holding_cost)) ** 0.5
eoq_inventory_cost = (engine_cost * holding_cost * eoq) / 2 + (daily_demand * truck_cost) / eoq

# Calculate the annual inventory cost with JIT load
jit_inventory_cost = (engine_cost * holding_cost * jit_load) / 2 + (daily_demand * truck_cost) / jit_load

# Calculate the difference in annual inventory costs
cost_difference = jit_inventory_cost - eoq_inventory_cost

print(f"The decision to reduce the number of engines loaded on each truck to 100 increases the annual inventory cost by ${cost_difference:.2f}.")

# Calculate the optimal truck cost for a load of 100 engines
optimal_truck_cost = (jit_load * engine_cost * holding_cost) / (2 * daily_demand)

print(f"For a load of 100 engines to be optimal, the cost of each truck should be ${optimal_truck_cost:.2f}.")

The decision to reduce the number of engines loaded on each truck to 100 increases the annual inventory cost by $254.03.
For a load of 100 engines to be optimal, the cost of each truck should be $16.67.


In [2]:
import math

# Given data
annual_demand = 300 * 365  # 300 motorcycles per day, assuming 365 days per year
ordering_cost = 1000  # $1,000 per truck trip
unit_cost = 500  # $500 per engine
holding_cost_rate = 0.2  # 20% annual holding cost rate
jit_load = 100  # JIT load of 100 engines per truck

# Calculate the annual holding cost per unit
annual_holding_cost = unit_cost * holding_cost_rate

# Calculate the Economic Order Quantity (EOQ)
eoq = math.sqrt((2 * annual_demand * ordering_cost) / annual_holding_cost)

# Calculate the total annual cost using EOQ
eoq_annual_ordering_cost = (annual_demand / eoq) * ordering_cost
eoq_annual_holding_cost = (eoq / 2) * annual_holding_cost
eoq_total_annual_cost = eoq_annual_ordering_cost + eoq_annual_holding_cost

# Calculate the total annual cost using JIT load
jit_annual_ordering_cost = (annual_demand / jit_load) * ordering_cost
jit_annual_holding_cost = (jit_load / 2) * annual_holding_cost
jit_total_annual_cost = jit_annual_ordering_cost + jit_annual_holding_cost

# Calculate the difference in annual costs
cost_difference = jit_total_annual_cost - eoq_total_annual_cost

print(f"The decision to reduce the number of engines loaded on each truck to 100 increases the annual inventory cost by ${cost_difference:.2f}.")

# Calculate the optimal truck cost for a load of 100 engines
optimal_truck_cost = (jit_load ** 2 * annual_holding_cost) / (2 * annual_demand)

print(f"For a load of 100 engines to be optimal, the cost of each truck should be ${optimal_truck_cost:.2f}.")

The decision to reduce the number of engines loaded on each truck to 100 increases the annual inventory cost by $952013.51.
For a load of 100 engines to be optimal, the cost of each truck should be $4.57.


## Problem 5: Green Thumb Manufacturing Decision

In [10]:
from scipy.stats import norm

# Given parameters
unit_cost = 150
selling_price = 200
mean_demand = 100
std_demand = 40
salvage_value = 50
holding_cost = 20

# Calculate the critical ratio
critical_ratio = (selling_price - unit_cost) / (selling_price - salvage_value)

# Calculate the optimal order quantity
z_value = norm.ppf(critical_ratio)
optimal_quantity = round(mean_demand + z_value * std_demand)

print(f"Green Thumb should manufacture {optimal_quantity} units for sale.")

# Calculate the expected profit
expected_sales = norm.cdf((optimal_quantity - mean_demand) / std_demand) * optimal_quantity + \
                 norm.pdf((optimal_quantity - mean_demand) / std_demand) * std_demand * (selling_price - unit_cost)
expected_salvage = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * (optimal_quantity - mean_demand) * salvage_value
expected_holding = (optimal_quantity - expected_sales) * holding_cost
expected_profit = expected_sales * (selling_price - unit_cost) + expected_salvage - expected_holding - optimal_quantity * unit_cost

print(f"The expected profit from this policy is ${expected_profit:.2f}.")

# Calculate the expected number of customers turned away
expected_lost_sales = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * mean_demand

print(f"On average, Green Thumb expects to turn away {expected_lost_sales:.2f} customers due to stockouts.")

Green Thumb should manufacture 83 units for sale.
The expected profit from this policy is $38302.75.
On average, Green Thumb expects to turn away 66.46 customers due to stockouts.


In [3]:
from scipy.stats import norm

# Given data
unit_cost = 150
selling_price = 200
mean_demand = 100
std_demand = 40
salvage_value = 50
holding_cost = 20

# Calculate the critical fractile
critical_fractile = (selling_price - unit_cost) / (selling_price - salvage_value)

# Calculate the optimal order quantity
z_value = norm.ppf(critical_fractile)
optimal_quantity = round(mean_demand + z_value * std_demand)

print(f"Green Thumb should manufacture {optimal_quantity} units for sale.")

# Calculate the expected profit
expected_sales = norm.cdf((optimal_quantity - mean_demand) / std_demand) * optimal_quantity + \
                 norm.pdf((optimal_quantity - mean_demand) / std_demand) * std_demand * (selling_price - unit_cost)
expected_salvage = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * (optimal_quantity - mean_demand) * salvage_value
expected_holding = (optimal_quantity - expected_sales) * holding_cost
expected_profit = expected_sales * (selling_price - unit_cost) + expected_salvage - expected_holding - optimal_quantity * unit_cost

print(f"The expected profit from this policy is ${expected_profit:.2f}.")

# Calculate the expected number of customers turned away
expected_lost_sales = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * mean_demand

print(f"On average, Green Thumb expects to turn away {expected_lost_sales:.2f} customers due to stockouts.")

Green Thumb should manufacture 83 units for sale.
The expected profit from this policy is $38302.75.
On average, Green Thumb expects to turn away 66.46 customers due to stockouts.


In [1]:
import numpy as np
from scipy.stats import norm

# Given data
unit_cost = 150
selling_price = 200
mean_demand = 100
std_demand = 40
salvage_value = 50
holding_cost = 20

# Calculate the critical fractile (optimal cycle service level)
critical_fractile = (selling_price - unit_cost) / (selling_price - salvage_value)

# Calculate the optimal order quantity
z_value = norm.ppf(critical_fractile)
optimal_quantity = round(mean_demand + z_value * std_demand)

print(f"Green Thumb should manufacture {optimal_quantity} units for sale.")

# Calculate the expected profit
expected_sales = norm.cdf((optimal_quantity - mean_demand) / std_demand) * optimal_quantity + \
                 norm.pdf((optimal_quantity - mean_demand) / std_demand) * std_demand * (selling_price - unit_cost)
expected_salvage = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * (optimal_quantity - mean_demand) * salvage_value
expected_holding = (optimal_quantity - expected_sales) * holding_cost
expected_profit = expected_sales * (selling_price - unit_cost) + expected_salvage - expected_holding - optimal_quantity * unit_cost

print(f"The expected profit from this policy is ${expected_profit:.2f}.")

# Simulate demand for n seasons to estimate the average number of customers turned away
n_seasons = 10000
simulated_demand = np.random.normal(mean_demand, std_demand, n_seasons)
stockouts = np.sum(simulated_demand > optimal_quantity)
average_lost_sales = stockouts / n_seasons

print(f"On average, Green Thumb expects to turn away {average_lost_sales:.2f} customers due to stockouts.")

Green Thumb should manufacture 83 units for sale.
The expected profit from this policy is $38302.75.
On average, Green Thumb expects to turn away 0.66 customers due to stockouts.


In [10]:
from scipy.stats import norm

# Given data
unit_cost = 150
selling_price = 200
mean_demand = 100
std_demand = 40
salvage_value = 50
holding_cost = 20

# Step 1: Calculate the critical fractile (optimal cycle service level)
critical_fractile = (selling_price - unit_cost) / (selling_price - salvage_value)

# Step 2: Calculate the optimal order quantity
z_value = norm.ppf(critical_fractile)
optimal_quantity = round(mean_demand + z_value * std_demand)

print(f"Green Thumb should manufacture {optimal_quantity} units for sale.")

# Step 3: Calculate the expected profit
expected_sales = norm.cdf((optimal_quantity - mean_demand) / std_demand) * optimal_quantity + \
                 norm.pdf((optimal_quantity - mean_demand) / std_demand) * std_demand * (selling_price - unit_cost)
expected_salvage = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * (optimal_quantity - mean_demand) * salvage_value
expected_holding = (optimal_quantity - expected_sales) * holding_cost
expected_profit = expected_sales * (selling_price - unit_cost) + expected_salvage - expected_holding - optimal_quantity * unit_cost

print(f"The expected profit from this policy is ${expected_profit:.2f}.")

# Step 4: Calculate the expected number of customers turned away (lost sales)
prob_demand_exceeds_optimal = 1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)
expected_lost_sales = (mean_demand - optimal_quantity) * prob_demand_exceeds_optimal

print(f"On average, Green Thumb expects to turn away {abs(expected_lost_sales):.2f} customers due to stockouts.")

Green Thumb should manufacture 83 units for sale.
The expected profit from this policy is $1218.40.
On average, Green Thumb expects to turn away 11.30 customers due to stockouts.


In [8]:
from scipy.stats import norm

# Given data
unit_cost = 150
selling_price = 200
mean_demand = 100
std_demand = 40
salvage_value = 50
holding_cost = 20

# Step 1: Calculate the critical fractile (optimal cycle service level)
critical_fractile = (selling_price - unit_cost) / (selling_price - salvage_value)

# Step 2: Calculate the optimal order quantity
z_value = norm.ppf(critical_fractile)
optimal_quantity = round(mean_demand + z_value * std_demand)

print(f"Green Thumb should manufacture {optimal_quantity} units for sale.")

# Step 3: Calculate the expected profit
expected_sales = norm.cdf((optimal_quantity - mean_demand) / std_demand) * optimal_quantity + \
                 norm.pdf((optimal_quantity - mean_demand) / std_demand) * std_demand * (selling_price - unit_cost)
print(f"expected_sales ${expected_sales:.2f}.")

expected_salvage = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * (optimal_quantity - mean_demand) * salvage_value
print(f"expected_salvage ${expected_salvage:.2f}.")

expected_holding = (optimal_quantity - expected_sales) * holding_cost
print(f"expected_holding ${expected_holding:.2f}.")

expected_profit = expected_sales  + expected_salvage - expected_holding - optimal_quantity * unit_cost

print(f"The expected profit from this policy is ${expected_profit:.2f}.")

# Step 4: Calculate the expected number of customers turned away (lost sales)
prob_demand_exceeds_optimal = 1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)
expected_lost_sales = (mean_demand - optimal_quantity) * prob_demand_exceeds_optimal

print(f"On average, Green Thumb expects to turn away {abs(expected_lost_sales):.2f} customers due to stockouts.")

Green Thumb should manufacture 83 units for sale.
expected_sales $756.82.
expected_salvage $-564.89.
expected_holding $-13476.47.
The expected profit from this policy is $1218.40.
On average, Green Thumb expects to turn away 11.30 customers due to stockouts.


In [11]:
from scipy.stats import norm

# Given data
unit_cost = 150
selling_price = 200
mean_demand = 100
std_demand = 40
salvage_value = 50
holding_cost = 20

# Step 1: Calculate the critical fractile (optimal cycle service level)
critical_fractile = (selling_price - unit_cost) / (selling_price - salvage_value)

# Step 2: Calculate the optimal order quantity
z_value = norm.ppf(critical_fractile)
optimal_quantity = round(mean_demand + z_value * std_demand)

print(f"Green Thumb should manufacture {optimal_quantity} units for sale.")

# Step 3: Calculate the expected profit
expected_sales_quantity = norm.cdf((optimal_quantity - mean_demand) / std_demand) * optimal_quantity + \
                          norm.pdf((optimal_quantity - mean_demand) / std_demand) * std_demand
expected_sales_revenue = expected_sales_quantity * selling_price
expected_salvage_revenue = (1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)) * (optimal_quantity - mean_demand) * salvage_value
expected_holding_cost = (optimal_quantity - expected_sales_quantity) * holding_cost
expected_profit = expected_sales_revenue + expected_salvage_revenue - expected_holding_cost - optimal_quantity * unit_cost

print(f"The expected profit from this policy is ${expected_profit:.2f}.")

# Step 4: Calculate the expected number of customers turned away (lost sales)
prob_demand_exceeds_optimal = 1 - norm.cdf((optimal_quantity - mean_demand) / std_demand)
expected_lost_sales = (mean_demand - optimal_quantity) * prob_demand_exceeds_optimal

print(f"On average, Green Thumb expects to turn away {abs(expected_lost_sales):.2f} customers due to stockouts.")

Green Thumb should manufacture 83 units for sale.
The expected profit from this policy is $-5342.63.
On average, Green Thumb expects to turn away 11.30 customers due to stockouts.
